# ejabT1E Package Overview

## What It Is

**ejabT1E** is an R package for detecting Type I errors in published NHST (Null Hypothesis Significance Testing) results using the *eJAB01* approximate objective Bayes factor.

The core idea: if a result is statistically significant (p <= alpha) but the Bayes factor strongly supports H0 (eJAB01 > C*), that result is a **Bayes/NHST contradiction** -- a candidate Type I error.

The package implements the full pipeline:
1. Compute eJAB01 for each result
2. Estimate the threshold C* via minimum-distance estimation
3. Flag contradictions (p <= alpha AND eJAB > C*)
4. Compute diagnostic U_i values (should be Unif(0,1) if assumptions hold)
5. Produce a diagnostic QQ-plot
6. Compute posterior P(T1E) for each candidate

## Package Functions

| Function | Purpose |
|----------|--------|
| `ejab01(p, n, q)` | Compute the eJAB01 Bayes factor |
| `objective_C(C, p, ejab, up)` | Closed-form objective for C* estimation |
| `estimate_Cstar(p, ejab, up, grid)` | Grid search for optimal C* |
| `detect_type1(p, ejab, alpha, Cstar)` | Flag Bayes/NHST contradictions |
| `diagnostic_U(p, n, q, alpha, Cstar)` | Compute diagnostic U_i values |
| `diagnostic_qqplot(U)` | QQ-plot of U_i vs Unif(0,1) |
| `posterior_t1e(p, ejab, alpha, all_objectives, grid)` | Posterior P(T1E) per result |
| `ejab_pipeline(df, up, alpha, grid, plot)` | Full pipeline wrapper |

---
## Function Details

### `ejab01(p, n, q)`

Computes the approximate objective Bayes factor (evidence for H0 over H1):

$$\text{eJAB01}_i = \sqrt{n_i} \cdot \exp\left( -\frac{1}{2} \cdot \frac{n_i^{1/q_i} - 1}{n_i^{1/q_i}} \cdot \chi^2_{q_i, 1-p_i} \right)$$

- `p`: p-values (0 < p < 1)
- `n`: sample sizes (must be > 1)
- `q`: test dimensions (number of parameters under H1)

In [ ]:
# Package code:
ejab01 <- function(p, n, q) {
  if (!all(p > 0 & p < 1)) stop("All p-values must be in (0, 1).")
  if (!all(n > 1)) stop("All sample sizes n must be > 1.")
  if (!all(q >= 1)) stop("All dimensions q must be >= 1.")
  sqrt(n) * exp(-0.5 * (n^(1/q) - 1) / n^(1/q) * qchisq(1 - p, df = q))
}

### `objective_C(C, p, ejab, up)`

The closed-form objective for estimating C*. Measures the integrated squared deviation between the empirical contradiction rate $\hat{P}(\alpha, C)$ and the theoretical T1E rate $\alpha / u_p$:

$$\text{obj}(C) = \sum_{j=1}^{J} (p_{(j+1)} - p_{(j)}) \hat{P}(p_{(j)}, C)^2 \;-\; \frac{1}{u_p N} \sum_{j=1}^{J} \xi_j^C (u_p^2 - p_{(j)}^2) \;+\; \frac{u_p}{3}$$

where:
- $p_{(j)}$ are sorted unique p-values, $p_{(J+1)} = u_p$
- $\xi_j^C$ = number of results at $p_{(j)}$ with eJAB > C ("multiplicities")
- $\hat{P}(p_{(j)}, C) = \frac{1}{N}\sum_{k=1}^{j} \xi_k^C$

In [ ]:
# Helper: compute multiplicities
compute_xi <- function(p_sub, ejab_sub, p_unique, C) {
  vapply(p_unique, function(pj) {
    sum(p_sub == pj & ejab_sub > C)
  }, numeric(1))
}

# Package code:
objective_C <- function(C, p, ejab, up) {
  idx <- p <= up
  p_sub <- p[idx]; ejab_sub <- ejab[idx]; N <- length(p_sub)
  if (N == 0) return(Inf)

  p_unique <- sort(unique(p_sub))
  xi_C <- compute_xi(p_sub, ejab_sub, p_unique, C)
  Phat_vals <- cumsum(xi_C) / N
  gaps <- diff(c(p_unique, up))

  term1 <- sum(gaps * Phat_vals^2)                          # integral of Phat^2
  term2 <- -sum(xi_C * (up^2 - p_unique^2)) / (up * N)     # cross term
  term3 <- up / 3                                           # constant
  term1 + term2 + term3
}

### `estimate_Cstar(p, ejab, up, grid)`

Grid search over C in [1/3, 3] (200 points). Returns the minimizer C* and the full vector of objective values (used later for posterior computation).

### `detect_type1(p, ejab, alpha, Cstar)`

Simple logical detection: a result is a candidate T1E if $p_i \le \alpha$ AND $\text{eJAB}_i > C^*$.

### `diagnostic_U(p, n, q, alpha, Cstar)`

For each candidate T1E, computes:

$$U_i = \frac{p_i - d_i}{\alpha - d_i}$$

where:

$$d_i = 1 - F_{\chi^2_{q_i}}\left( \frac{2 n_i^{1/q_i}}{n_i^{1/q_i} - 1} \log\frac{\sqrt{n_i}}{C^*} \right)$$

Under the left-tail uniformity assumption, $U_i \sim \text{Unif}(0,1)$.

In [ ]:
# Package code:
diagnostic_U <- function(p, n, q, alpha, Cstar) {
  if (any(n <= 1)) stop("Sample sizes n must be > 1 for diagnostic computation.")
  d <- 1 - pchisq(
    (2 * n^(1/q) / (n^(1/q) - 1)) * log(sqrt(n) / Cstar),
    df = q
  )
  (p - d) / (alpha - d)
}

### `posterior_t1e(p, ejab, alpha, all_objectives, grid)`

Treats the objective as a negative log-likelihood. Under a uniform prior on C:

$$\pi(C_k \mid \text{data}) \propto \exp\{-\text{obj}(C_k)\}$$

For each result $i$, the posterior probability of being a T1E is:

$$P(\text{T1E}_i) = \sum_{k:\, p_i \le \alpha,\; \text{eJAB}_i > C_k} \pi(C_k \mid \text{data})$$

This sums posterior mass over all C values where that result would be flagged.

### `ejab_pipeline(df, up, alpha, grid, plot)`

Full wrapper. Expects a data frame with columns `p`, `n`, `q` (and optional `ID`). Runs all steps and returns C*, candidates with posterior probabilities, diagnostics, and the posterior over C.

Input validation:
- `alpha <= up` (required by theory)
- `n > 1` (required by diagnostic formula)
- Required columns exist

---
## Changes from `newMethod.R`

The package was rewritten from the original `newMethod.R` prototype to fix mathematical errors and add missing functionality. Below is a detailed comparison.

### 1. `ejab01` -- Minor Fix

The exponent term was missing a division by $n^{1/q}$.

| | Expression |
|---|---|
| **Original** | `exp(-0.5 * (n^(1/q) - 1) * chi_quant)` |
| **Corrected** | `exp(-0.5 * (n^(1/q) - 1) / n^(1/q) * chi_quant)` |

The correct formula has $\frac{n^{1/q} - 1}{n^{1/q}}$ not just $(n^{1/q} - 1)$.

In [ ]:
# ORIGINAL (newMethod.R):
# sqrt(n) * exp(-0.5 * (n^(1/q) - 1) * chi_quant)

# CORRECTED (Package):
# sqrt(n) * exp(-0.5 * (n^(1/q) - 1) / n^(1/q) * qchisq(1 - p, df = q))

### 2. `objective_C` -- Major Rewrite

The original used an **unweighted pointwise sum** of squared differences, which does not match the closed-form integrated objective from the theory.

**Original approach** (wrong):
- Looped over unique p-values, computing `phat(...)` at each
- Accumulated `(phat - alpha/up)^2` with no gap weighting
- This is a Riemann sum with equal weights -- incorrect because the p-value spacing is non-uniform

**Corrected approach:**
- Computes multiplicities $\xi_j^C$ (count of results at each unique p-value with eJAB > C)
- Uses the closed-form with gap-weighted $\hat{P}^2$ term, cross term using $(u_p^2 - p_{(j)}^2)$, and constant $u_p/3$
- Properly reflects the integral $\int_0^{u_p} [\hat{P}(\alpha, C) - \alpha/u_p]^2 \, d\alpha$

In [ ]:
# ORIGINAL (newMethod.R):
# objective_C <- function(C, p, ejab, up) {
#   p_sub <- p[p <= up]
#   p_grid <- sort(unique(p_sub))
#   obj <- 0
#   for (j in seq_along(p_grid)) {
#     alpha <- p_grid[j]
#     ph <- phat(p, ejab, alpha, C, up)
#     obj <- obj + (ph - alpha / up)^2     <-- unweighted sum, wrong
#   }
#   obj
# }

# CORRECTED: uses closed-form with gap weighting, multiplicities, cross term

### 3. `diagnostic_U` -- Sign Error in Log

The `d_i` computation had `log(sqrt(n) * Cstar)` (multiplication) instead of `log(sqrt(n) / Cstar)` (division).

| | Expression inside log |
|---|---|
| **Original** | `log(sqrt(ni) * Cstar)` |
| **Corrected** | `log(sqrt(n) / Cstar)` |

This corresponds to the theory: $d_i = 1 - F_{\chi^2_q}\left(\frac{2n^{1/q}}{n^{1/q}-1} \log\frac{\sqrt{n}}{C^*}\right)$

In [ ]:
# ORIGINAL (newMethod.R):
# (2 * ni^(1/qi) / (ni^(1/qi) - 1)) * log(sqrt(ni) * Cstar)   <-- WRONG: multiply

# CORRECTED (Package):
# (2 * n^(1/q) / (n^(1/q) - 1)) * log(sqrt(n) / Cstar)        <-- CORRECT: divide

### 4. Grid Range -- Updated

| | Grid |
|---|---|
| **Original** | `seq(1, 3, length.out = 200)` |
| **Corrected** | `seq(1/3, 3, length.out = 200)` |

The lower bound was changed from 1 to 1/3 per discussion with Dr. Nathoo (Jan 16), to provide better sensitivity at low effect sizes where C* < 1.

### 5. New Functionality (Not in Original)

| Feature | Description |
|---------|-------------|
| `posterior_t1e()` | Computes P(T1E) for each candidate by treating C* as a posterior mode |
| `diagnostic_qqplot()` | QQ-plot of U_i vs Unif(0,1) |
| Input validation | Guards for `p in (0,1)`, `n > 1`, `q >= 1`, `alpha <= up` |
| `all_objectives` in output | Returns full objective vector for posterior computation |
| `posterior_C` in output | Returns posterior distribution over C grid |
| ID column preservation | Pipeline preserves optional `ID` column in candidates output |

### 6. Structural / Engineering Changes

| Aspect | Original | Package |
|--------|----------|--------|
| Format | Single script file | Proper R package (DESCRIPTION, NAMESPACE, man/, tests/) |
| Column name | `df$N` (capital) | `df$n` (lowercase, consistent with function args) |
| Vectorization | `sapply` loop in `diagnostic_U` | Fully vectorized |
| Documentation | None | Roxygen-style comments + .Rd man pages |
| Tests | None | 8 unit tests in `tests/test_ejab.R` |
| CRAN | N/A | Passes `R CMD check` with Status: OK |

---
## Summary of Bug Impact

1. **ejab01 exponent**: Missing `/n^(1/q)` would overstate the Bayes factor for large n, causing fewer detections than appropriate.

2. **objective_C unweighted sum**: Would produce a biased C* estimate. The correct closed-form accounts for non-uniform p-value spacing via gap weighting.

3. **diagnostic_U multiply vs divide**: With `*` instead of `/`, $d_i$ would be computed at the wrong chi-squared quantile, invalidating the Unif(0,1) calibration check entirely.

---
## Usage Example

```r
library(ejabT1E)

df <- data.frame(
  ID = 1:100,
  p = runif(100, 0, 0.05),
  n = sample(20:200, 100, replace = TRUE),
  q = rep(1, 100)
)

result <- ejab_pipeline(df, up = 0.05, alpha = 0.05)

result$Cstar                    # estimated threshold
result$candidates               # flagged T1Es with posterior probs
result$posterior_C              # posterior distribution over C
```

---

---

---

- [ ] Double check the funding ($1250) and scholarship, this is of SC(something? data analysis)
- [ ] Hackathon: Check airfare!!! Check dates! Etc.
- [ ] Add R oxygen to package folder
- R package:
  - Minimizer: Make the search space from 0-10
  - In sim studies, we want to demonstrate the method works & investigate how good $C^*$ is relative to $1$
  - Thus we should compare $1$ and the AVG $C^*$

### posterior_t1e

QUestion: What' the prior probability of a T1E?
- It is .05

We put a uniform prior on C; we dont know if its good or nor
- We can potentially test like this:
  - We get a posterior
  - Add posterior corresponding to values of $C$ that make us declare a T1E
  - What if we did this with the prior as well?
    - Grid of $C$ values
      - 5 of which ejab is greater than
      - Add posterior prob of those 5 to get posterior prob of a t1e (this is what it does)
    - Then: Sum over all C's on grid where p \leq apha and ejab > C
      - This sum is the posterior prob
    - For the same C's, sum the prior probabilities (1/k)
      - Can we turn this into a bayes factor?
  - The issue being conidere ia that we have done normalizaiton to get probabilities which assumes a uniform prior - is this a good idea? What effect does it have on results?
  - How should alpha (the prior probability) come into the answer? How do we get a posterior from this?
- Prof assumes probabilities will increase 

---

- ROC curves before were great: We will do same, but with C=1 and C^*
- In a more realistic simulation:
  - Randomly draw es
  - Randomly draw sample size
  - Incorporate selection process
    - Only retain smaller p-values
      - All p-values less than .05 are retained/kept
      - p-values greater than .05 are almost never included; diminish w size of p-value
      - Keep p-val in sample w a probability 1-p
- Keep it simple: 
  - Instead of randomly generating es, ss, the only thing we add is a selection process
- We will do a set of sims with a selection process, and one without
  - Should we do it for everything? (Yes!)
  - T-tetss first. Then all from original paper